In [ ]:
import nibabel as nib
import pandas as pd
import numpy as np
import os
import scipy.io as sio

In [3]:
def cifti2_filename(cifti1_path, output_dir):
    basename = os.path.basename(cifti1_path)

    # List of known CIFTI suffixes
    known_suffixes = [".dtseries.nii", ".dconn.nii", ".dscalar.nii", ".dpconn.nii"]

    # Match and strip full suffix
    for suffix in known_suffixes:
        if basename.endswith(suffix):
            stem = basename[: -len(suffix)]  # strip the full suffix
            cifti2_name = f"{stem}_cifti2{suffix}"
            return os.path.join(output_dir, cifti2_name)

    raise ValueError("Unknown CIFTI suffix in filename")

In [ ]:
def calc_correlations(input_file, tmask):
    if input_file.endswith(".dtseries.nii"):
        cifti_img = nib.load(input_file)
        cifti_data = cifti_img.get_fdata()
        # cifti_header = cifti_img.header
        bm_index_map = cifti_img.header.get_index_map(1)

        # Initialize storage
        vertex_indices = []
        # Loop through the brain models in the index map
        for bm in bm_index_map.brain_models:
            structure = bm.brain_structure
            if structure in ['CIFTI_STRUCTURE_CORTEX_LEFT', 'CIFTI_STRUCTURE_CORTEX_RIGHT']:
                # Convert Cifti2VertexIndices to numpy array
                offset = bm.index_offset
                count = bm.index_count
                vertex_indices.extend(range(offset, offset + count))

        vertex_indices = np.array(vertex_indices)
        cortex_ts = np.transpose(cifti_data[:, vertex_indices])
        masked_cortex_ts = cortex_ts[:, tmask]
        corr_matrix = np.corrcoef(masked_cortex_ts)
        return corr_matrix
    else:
        raise ValueError("Input file must be a CIFTI-2 time-series file. ")

In [ ]:
def format_data(corr_vec, transmat_path, img_size, rand_permutation, interval):
    assert corr_vec.shape == (59412, )
    left_data, right_data = corr_vec[:29697], corr_vec[29697:]
    left_transmat = sio.loadmat(os.path.join(transmat_path, "Left_fMRI2Grid_192_by_192_NN.mat"))
    right_transmat = sio.loadmat(os.path.join(transmat_path, "Right_fMRI2Grid_192_by_192_NN.mat"))
    


In [ ]:
example_cifti_cohort_filepath = "./data/cohort_files/cohortfiles_washu120.txt"
example_tmask_cohort_filepath = "./data/tmask_files/tmasklist_washu120.txt"

cifti_cohort_df = pd.read_csv(example_cifti_cohort_filepath, delim_whitespace=True, header=None)
tmask_cohort_df = pd.read_csv(example_tmask_cohort_filepath, delim_whitespace=True, header=None)

example_idx = 111
subj, cifti1_path = cifti_cohort_df.iloc[example_idx, :2].tolist()
tmask_subj, tmask_path = tmask_cohort_df.iloc[example_idx, :].tolist()
assert subj == tmask_subj
tmask = np.loadtxt(tmask_path, dtype=int).astype(bool)

cifti2_path = cifti2_filename(cifti1_path, "./data/washu120/")

cifti_img = nib.load(cifti2_path)
cifti_data = cifti_img.get_fdata()
cifti_header = cifti_img.header
bm_index_map = cifti_img.header.get_index_map(1)

# Initialize storage
vertex_indices = []
# Loop through the brain models in the index map
for bm in bm_index_map.brain_models:
    structure = bm.brain_structure
    if structure in ['CIFTI_STRUCTURE_CORTEX_LEFT', 'CIFTI_STRUCTURE_CORTEX_RIGHT']:
        # Convert Cifti2VertexIndices to numpy array
        offset = bm.index_offset
        count = bm.index_count
        vertex_indices.extend(range(offset, offset + count))

vertex_indices = np.array(vertex_indices)
cortex_ts = np.transpose(cifti_data[:, vertex_indices])
masked_cortex_ts = cortex_ts[:, tmask]
corr_matrix = np.corrcoef(masked_cortex_ts)
# print(corr_matrix.shape)

(59412, 59412)
